# Talk To Contract Agent - Testing and Examples

This notebook demonstrates how to use the Talk To Contract agent with various filter configurations.

## Setup and Imports

In [1]:
from contramate.core.agents import (
    TalkToContractAgentFactory,
    TalkToContractDependencies,
)
from contramate.services.opensearch_vector_search_service import (
    OpenSearchVectorSearchServiceFactory,
)
from pathlib import Path
from IPython.display import Markdown, display
from contramate.models import MessageHistory

2025-10-24 01:37:33.813 | INFO     | contramate.utils.settings.base:find_env_file_if_exists:25 - Loading settings from System Environment


In [2]:
ENV_FILE = Path().absolute().parent / ".envs" / "local.env"
print(f"Loading environment variables from: {ENV_FILE}")

Loading environment variables from: /Users/datapsycho/PythonProjects/AgentEngBootCamp/contramate/.envs/local.env


## Initialize Agent and Search Service

In [3]:
# Create agent and search service
agent = TalkToContractAgentFactory.from_env_file(env_path=ENV_FILE)
search_service = OpenSearchVectorSearchServiceFactory.from_env_file(env_path=ENV_FILE)
print("✅ Agent and search service initialized")

2025-10-24 01:37:34.255 | INFO     | contramate.llm.factory:get_default_client:81 - Creating vanilla async client for provider: openai
2025-10-24 01:37:34.316 | INFO     | contramate.integrations.aws.opensearch:create_opensearch_client:52 - Created OpenSearch client for localhost:9200


✅ Agent and search service initialized


In [4]:
# search_response = search_service.hybrid_search("what is the payment terms?")
# result = search_response.unwrap()

In [5]:
# display(Markdown(result.to_llm_context()))

## Example 1: Query Without Filters

Search across all documents (with default `doc_source=system` filter)

In [14]:
# Create dependencies without filters
deps = TalkToContractDependencies(search_service=search_service)

# Run query
user_query = "What are common payment terms in contracts?"
result = await agent.run(user_query, deps=deps)

# Display results
print(f"\n{'='*80}")
print(f"Query: {user_query}")
print(f"{'='*80}")
print(f"\nAnswer:\n{result.output.answer}")
print(f"\n{'='*80}")
print("Citations:")
print(f"{'='*80}")
for key, value in result.output.citations.items():
    print(f"  {key}: {value}")

2025-10-23 13:38:45 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-10-23 13:38:45.232 | INFO     | contramate.core.agents.talk_to_contract:hybrid_search:491 - 🔍 Tool: hybrid_search called with query='common payment terms in contracts...'
2025-10-23 13:38:45.232 | INFO     | contramate.services.opensearch_vector_search_service:hybrid_search:449 - 🔍 Performing hybrid search for: 'common payment terms in contracts...'
2025-10-23 13:38:45 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-23 13:38:45.461 | INFO     | contramate.services.opensearch_vector_search_service:hybrid_search:455 - ✅ Generated embedding vector with 1536 dimensions
2025-10-23 13:38:45 - opensearch - INFO - POST http://localhost:9200/contracts-v1/_search [status:200 request:0.117s]
2025-10-23 13:38:45.581 | INFO     | contramate.services.opensearch_vector_search_service:hybrid_search:547 - ✅ Hybrid search returned 10


Query: What are common payment terms in contracts?

Answer:
Common payment terms in contracts typically include the following:

- Payment is often due within a specified period after invoice receipt, commonly net 30 days. For example, in manufacturing outsourcing agreements, payment by the client is due thirty (30) days from the date of invoice issuance [doc1].

- In transportation service agreements, payment is generally due within 15 days of the invoice date, with late payments accruing interest at a specified rate (e.g., 2% per annum) [doc2].

- Consulting agreements may specify payment in equal quarterly installments, such as an annual consulting fee paid in four equal quarterly payments on specified dates [doc3][doc4].

- Supply agreements often require payment in U.S. dollars within 30 days after the date of shipment, with provisions for advance payment or modification of payment terms in case of changes in circumstances [doc6].

- Reseller agreements commonly require invoices t

## Example 2: Single Document Filter

Limit search to a specific document

In [ ]:
{ 
    "query": "What are the payment terms?", 
    "filters": { 
        "documents": [ 
            { 
                "project_id": "00149794-2432-4c18-b491-73d0fafd3efd", 
                "reference_doc_id": "577ff0a3-a032-5e23-bde3-0b6179e97949" 
            } 
        ] 
    }, 
    "message_history": [
        {"role": "user", "content": "Hello"},
        {"role": "assistant", "content": "Hi! How can I help?"}
    ]
}

In [4]:
message_history =  {
    "messages": [
        {"role": "user", "content": "Hello"},
        {"role": "assistant", "content": "Hi! How can I help?"}
    ]
}
message_history_model = MessageHistory.model_validate(message_history)
pydantic_message_history = message_history_model.to_pydantic_ai_messages()

In [5]:
pydantic_message_history

[ModelRequest(parts=[UserPromptPart(content='Hello', timestamp=datetime.datetime(2025, 10, 23, 23, 37, 41, 395299, tzinfo=datetime.timezone.utc))]),
 ModelResponse(parts=[TextPart(content='Hi! How can I help?')], usage=RequestUsage(), timestamp=datetime.datetime(2025, 10, 23, 23, 37, 41, 395316, tzinfo=datetime.timezone.utc))]

In [ ]:
from pydantic_ai import ModelRequest, ModelResponse, UserPromptPart, TextPart
import datetime

In [9]:
# Convert raw history to Pydantic AI messages
history = [
    ModelRequest(parts=[
        UserPromptPart(content=message_history["messages"][0]['content'], timestamp=datetime.datetime.now())
    ]),
    ModelResponse(parts=[
        TextPart(content=message_history["messages"][1]['content'])
    ])
]


In [10]:
# Define single document filter
filter_config = {
    "documents": [
        {
            "project_id": "00149794-2432-4c18-b491-73d0fafd3efd",
            "reference_doc_id": "577ff0a3-a032-5e23-bde3-0b6179e97949",
        }
    ]
}



# Create dependencies with filter
deps_single = TalkToContractDependencies(
    search_service=search_service,
    filters=filter_config
)

# Run query
user_query = "What are the payment terms?"
result = await agent.run(user_query, deps=deps_single, message_history=history)


print(f"\nAnswer:\n{result.output.answer}")
print(f"\n{'='*80}")
print("Citations:")
print(f"{'='*80}")
print(f"{result.output.citations}")

2025-10-24 01:41:54 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-10-24 01:41:54.499 | INFO     | contramate.core.agents.talk_to_contract:hybrid_search:683 - 🔍 Tool: hybrid_search called with query='payment terms...'
2025-10-24 01:41:54.499 | INFO     | contramate.services.opensearch_vector_search_service:hybrid_search:449 - 🔍 Performing hybrid search for: 'payment terms...'
2025-10-24 01:41:54 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-24 01:41:54.788 | INFO     | contramate.services.opensearch_vector_search_service:hybrid_search:455 - ✅ Generated embedding vector with 1536 dimensions
2025-10-24 01:41:54 - opensearch - INFO - POST http://localhost:9200/contracts-v1/_search [status:200 request:0.088s]
2025-10-24 01:41:54.878 | INFO     | contramate.services.opensearch_vector_search_service:hybrid_search:547 - ✅ Hybrid search returned 6 results
2025-10-24 01:42:00 - httpx - IN


Answer:
The payment terms outlined in the agreement are as follows:

- The total price payable by the Publishers is set out in Clause 19.2 and the Use Fees in Clause 19.4, subject to the terms and conditions in the Agreement. This price is a fixed price.
- HealthGate shall invoice the Publishers for payment according to a schedule in Clause 19.2, which includes specific payment amounts and dates such as $100,000 on 30 January 1998, $150,000 on 6 February 1998, and subsequent payments on acceptance of specification, system launch, system completion date, and quarterly payments in 1999.
- Invoices are payable within 60 days of receipt, except for payments due under Clause 19.2, which are payable on the due date or on acceptance of the work, whichever is later.
- Use Fees are payments made by the Publishers to HealthGate based on "Use" of the Content, defined as retrieval or download of full-text articles by subscribers. Use Fees are billed monthly and payments are due by cheque by the e

In [9]:
result.new_messages()

[ModelRequest(parts=[SystemPromptPart(content='\n## Role & Context\n\nYou are a procurement assistant. You specialize in answering questions about contractual documents, supplier agreements, and procurement processes using advanced vector search capabilities. Your responses must be accurate, well-cited, and based exclusively on available data sources.\n\n## Search Result Context Format\n\nWhen you receive search results, they will be formatted as structured context with the following format:\n\n```markdown\n# Search Results for: [query]\n**Total Results:** X (of Y total)\n**Search Type:** [hybrid/semantic/text]\n\n---\n\n# Search Result 1\n\n| Field | Value |\n|-------|-------|\n| Document | [display_name] |\n| Contract Type | [contract_type] |\n| Section | [section_hierarchy] |\n\n**Content:**\n\n[actual content text...]\n\n---\n\n# Search Result 2\n\n[similar format...]\n```\n\n**IMPORTANT CITATION MAPPING:**\n- **Citations are INDEPENDENT of the "# Search Result N" numbering**\n- Yo

## Example 3: Multiple Documents Filter

Compare information across specific documents

In [4]:
# Define multiple documents filter
filter_config_multi = {
    "documents": [
        {
            "project_id": "00149794-2432-4c18-b491-73d0fafd3efd",
            "reference_doc_id": "577ff0a3-a032-5e23-bde3-0b6179e97949",
        },
        {
            "project_id": "008a9fd2-9a4a-4c3f-ad5c-d33eca94af3b",
            "reference_doc_id": "aa1a0c65-8016-5d11-bbde-22055140660b",
        },
        {
            "project_id": "0096b72f-1c0d-4724-924f-011f87d3591a",
            "reference_doc_id": "16b6078b-248c-5ed9-83ef-20ee0af49396",
        },
    ]
}

# Create dependencies with filter
deps_multi = TalkToContractDependencies(
    search_service=search_service,
    filters=filter_config_multi
)

# Run query
user_query = "Compare the liability limitations across these contracts"
result = await agent.run(user_query, deps=deps_multi)

# Display results
print(f"\n{'='*80}")
print(f"Query: {user_query}")
print(f"Filter: {len(filter_config_multi['documents'])} documents")
print(f"{'='*80}")
print(f"\nAnswer:\n{result.output.answer}")
print(f"\n{'='*80}")
print("Citations:")
print(f"{'='*80}")
for key, value in result.output.citations.items():
    print(f"  {key}: {value}")

2025-10-23 14:31:46 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-10-23 14:31:46.277 | INFO     | contramate.core.agents.talk_to_contract:compare_filtered_documents:556 - 🔍 Tool: compare_filtered_documents called with query='liability limitations...'
2025-10-23 14:31:46.278 | INFO     | contramate.core.agents.talk_to_contract:compare_filtered_documents:573 - 📋 Comparing across 3 filtered documents using optimized search
2025-10-23 14:31:46.279 | INFO     | contramate.services.opensearch_vector_search_service:hybrid_search_multi_document:905 - 🔍 Generating embedding once for multi-document search: 'liability limitations...'
2025-10-23 14:31:46 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-23 14:31:46.686 | INFO     | contramate.services.opensearch_vector_search_service:hybrid_search_multi_document:912 - ✅ Generated embedding vector with 1536 dimensions
2025-10-23 14:31:46.687 | I


Query: Compare the liability limitations across these contracts
Filter: 3 documents

Answer:
The liability limitations in the Hosting and Management Agreement with HealthGate Data Corp. specify that both parties agree to indemnify each other against claims related to death, bodily injury, or property damage caused by their tortious acts or omissions. They also indemnify each other against claims of intellectual property infringement related to the system. Except for personal injury or death caused by negligence, the liability of either party for any claims arising from performance or nonperformance under the agreement is limited to the amount of payments made under the agreement [doc1].

In the Manufacturing, Design and Marketing Agreement between Zounds Hearing, Inc. and InnerScope Hearing Technologies, Inc., neither party shall be liable to the other for any "cover" damages, including internal cover damages, or any incidental, consequential, special, or punitive damages arising out 

# Document Check Missing

In [10]:
doc3_liability = search_service.hybrid_search(
    query="liability limitations indemnification",
    filters={
        "documents": [{
            "project_id": "0096b72f-1c0d-4724-924f-011f87d3591a",
            "reference_doc_id": "16b6078b-248c-5ed9-83ef-20ee0af49396"
        }]
    },
    size=5
)

if doc3_liability.is_ok():
    response = doc3_liability.unwrap()
    print(f"Liability search results: {len(response.results)}")
    if response.results:
        for i, result in enumerate(response.results, 1):
            print(f"\n--- Result {i} (score: {result.score}) ---")
            print(result.content[:500])
    else:
        print("No results found for liability-related content")
        print("\nLet's see what IS in this document:")
        # Get the full chunk
        full_doc = search_service.search_by_document(
            project_id="0096b72f-1c0d-4724-924f-011f87d3591a",
            reference_doc_id="16b6078b-248c-5ed9-83ef-20ee0af49396"
        )
        if full_doc.is_ok():
            print(f"\nFull document content ({full_doc.unwrap().results[0].token_count} tokens):")
            print(full_doc.unwrap().results[0].content)
else:
    print(f"Error: {doc3_liability.unwrap_err()}")

2025-10-23 14:00:42.807 | INFO     | contramate.services.opensearch_vector_search_service:hybrid_search:449 - 🔍 Performing hybrid search for: 'liability limitations indemnification...'
2025-10-23 14:00:43 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-23 14:00:43.157 | INFO     | contramate.services.opensearch_vector_search_service:hybrid_search:455 - ✅ Generated embedding vector with 1536 dimensions
2025-10-23 14:00:43 - opensearch - INFO - POST http://localhost:9200/contracts-v1/_search [status:200 request:0.061s]
2025-10-23 14:00:43.221 | INFO     | contramate.services.opensearch_vector_search_service:hybrid_search:547 - ✅ Hybrid search returned 0 results
2025-10-23 14:00:43 - opensearch - INFO - POST http://localhost:9200/contracts-v1/_search [status:200 request:0.014s]
2025-10-23 14:00:43.237 | INFO     | contramate.services.opensearch_vector_search_service:search_by_document:821 - ✅ Retrieved 1 chunks from document 16b6078b-248c

Liability search results: 0
No results found for liability-related content

Let's see what IS in this document:

Full document content (474 tokens):
**AMENDMENT NO. 3 TO STREMICK’S HERITAGE FOODS, LLC and PREMIER NUTRITION CORPORATION MANUFACTURING AGREEMENT**

This Amendment No. 3 (the “Third Amendment”), entered into by and between Stremicks Heritage Foods, LLC (“Heritage”) Premier Nutrition
Corporation (“Premier”) is effective as of July 3, 2019 (“Third Amendment Effective Date”) and amends that certain Manufacturing Agreement between
Heritage and Premier dated July 1, 2017 as amended (“Agreement”). Heritage and Premier are each referred to herein as a “Party” and collectively as the
“Parties.”

**WHEREAS,** Heritage and Premier entered into the Agreement;

**WHEREAS,** the Parties wish to amend the Agreement in accordance with the terms and conditions set forth herein.

**NOW, THEREFORE,** in consideration of the promises and of the mutual covenants, representations and warranties 

## Example 4: Project Filter

Search within a specific project

In [ ]:
# Define project filter
filter_config_project = {
    "project_id": ["00149794-2432-4c18-b491-73d0fafd3efd"]
}

# Create dependencies with filter
deps_project = TalkToContractDependencies(
    search_service=search_service,
    filters=filter_config_project
)

# Run query
user_query = "What are the key obligations in project contracts?"
result = await agent.run(user_query, deps=deps_project)

# Display results
print(f"\n{'='*80}")
print(f"Query: {user_query}")
print(f"Filter: Project scope")
print(f"{'='*80}")
print(f"\nAnswer:\n{result.output.answer}")
print(f"\n{'='*80}")
print("Citations:")
print(f"{'='*80}")
for key, value in result.output.citations.items():
    print(f"  {key}: {value}")

## Example 5: Combined Filters

Use multiple filter types together

In [ ]:
# Define combined filters
filter_config_combined = {
    "project_id": ["00149794-2432-4c18-b491-73d0fafd3efd"],
    "doc_source": "system",
    "contract_type": ["Service Agreement", "License Agreement"]
}

# Create dependencies with filter
deps_combined = TalkToContractDependencies(
    search_service=search_service,
    filters=filter_config_combined
)

# Run query
user_query = "What are the warranty clauses?"
result = await agent.run(user_query, deps=deps_combined)

# Display results
print(f"\n{'='*80}")
print(f"Query: {user_query}")
print(f"Filter: Project + Source + Contract Type")
print(f"{'='*80}")
print(f"\nAnswer:\n{result.output.answer}")
print(f"\n{'='*80}")
print("Citations:")
print(f"{'='*80}")
for key, value in result.output.citations.items():
    print(f"  {key}: {value}")

## Inspecting the Response Structure

In [ ]:
# Show the complete response structure
print("Response fields:")
print(f"  - answer: {type(result.output.answer)}")
print(f"  - citations: {type(result.output.citations)}")
print(f"\nCitations dictionary:")
import json
print(json.dumps(result.output.citations, indent=2))

## Custom Query - Try Your Own

Use this cell to test custom queries with different filters

In [ ]:
# Define your custom filter (or set to None for no filter)
my_filter = {
    "documents": [
        {
            "project_id": "00149794-2432-4c18-b491-73d0fafd3efd",
            "reference_doc_id": "577ff0a3-a032-5e23-bde3-0b6179e97949",
        }
    ]
}

# Or set to None for no filter:
# my_filter = None

# Create dependencies
deps_custom = TalkToContractDependencies(
    search_service=search_service,
    filters=my_filter
)

# Your custom query
my_query = "What are the termination conditions?"

# Run query
result = await agent.run(my_query, deps=deps_custom)

# Display results
print(f"\n{'='*80}")
print(f"Query: {my_query}")
print(f"{'='*80}")
print(f"\nAnswer:\n{result.output.answer}")
print(f"\n{'='*80}")
print("Citations:")
print(f"{'='*80}")
for key, value in result.output.citations.items():
    print(f"  {key}: {value}")